In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
spandan2_cats_faces_64x64_for_generative_models_path = kagglehub.dataset_download('spandan2/cats-faces-64x64-for-generative-models')

print('Data source import complete.')


Data source import complete.


In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential
import kagglehub
import cv2
import os

# Set the random seed for reproducibility
tf.random.set_seed(42)

# Download the dataset
path = kagglehub.dataset_download("spandan2/cats-faces-64x64-for-generative-models")

# Load and preprocess the dataset
def load_images(data_path):
    images = []
    for filename in os.listdir(data_path):
        img_path = os.path.join(data_path, filename)
        for file in os.listdir(img_path):
          img = cv2.imread(img_path + '/' + file)
          if img is not None:  # Check if image loaded successfully
              img = cv2.resize(img, (64, 64))  # Resize to 64x64
              img = (img.astype(np.float32) - 127.5) / 127.5  # Scale to [-1, 1]
              images.append(img)
          else:
              pass
    return np.array(images)

# Load images into a NumPy array
cat_images = load_images(path)
print(f"Loaded {cat_images.shape[0]} images of shape {cat_images.shape[1:]}")

Loaded 15747 images of shape (64, 64, 3)


In [4]:
# Function to generate and save images
def generate_and_save_images(generator, epoch):
    noise = np.random.normal(0, 1, (25, LATENT_DIM))  # Generate 25 random noise vectors
    generated_images = generator.predict(noise)

    # Rescale images from [-1, 1] to [0, 1] for display
    generated_images = 0.5 * generated_images + 0.5

    # Plot the generated images
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.imshow(generated_images[i])
        plt.axis("off")
    plt.tight_layout()
    plt.savefig(f"gan_generated.png")  # Save the generated images
    plt.close()

In [7]:


# Build the generator model
def build_generator():
    model = Sequential()
    model.add(Dense(128 * 16 * 16, activation="relu", input_shape=(100,)))  # Latent dim
    model.add(Reshape((16, 16, 128)))  # Correctly reshape to (16, 16, 128)
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64 * 64 * 3, activation="tanh"))  # Output size for 64x64 RGB images
    model.add(Reshape((64, 64, 3)))  # Reshape to (64, 64, 3)
    return model

# Build the discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(64, 64, 3)))
    model.add(Dense(128, activation=LeakyReLU(0.2)))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="sigmoid"))  # Output layer for binary classification
    return model

# Create the models
generator = build_generator()
discriminator = build_discriminator()

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Build the GAN model
discriminator.trainable = False  # Freeze the discriminator while training GAN
gan_input = tf.keras.Input(shape=(100,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)

gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Function to train the GAN
def train_gan(gan, generator, discriminator, dataset, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        # Train the Discriminator
        idx = np.random.randint(0, dataset.shape[0], batch_size)
        real_images = dataset[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)

        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print the progress
        if epoch:
            print("Epoch: {}, Discriminator Loss: {:.4f}, Generator Loss: {:.4f}".format(epoch, d_loss[0], g_loss))
            generate_and_save_images(generator, epoch)

# Hyperparameters
LATENT_DIM = 100
EPOCHS = 1000
BATCH_SIZE = 64

# Train the GAN
train_gan(gan, generator, discriminator, cat_images, LATENT_DIM, EPOCHS, BATCH_SIZE)

# Generate and save images after training
generate_and_save_images(generator, EPOCHS)


2/2 [==============================] - 0s 77ms/step
Epoch: 1, Discriminator Loss: 2.7980, Generator Loss: 32.4923
2/2 [==============================] - 0s 76ms/step
Epoch: 2, Discriminator Loss: 1.2922, Generator Loss: 16.4729
2/2 [==============================] - 0s 66ms/step
Epoch: 3, Discriminator Loss: 1.8004, Generator Loss: 16.3040
2/2 [==============================] - 0s 65ms/step
Epoch: 4, Discriminator Loss: 0.7680, Generator Loss: 14.9946
2/2 [==============================] - 0s 73ms/step
Epoch: 5, Discriminator Loss: 0.8276, Generator Loss: 6.3774
2/2 [==============================] - 0s 67ms/step
Epoch: 6, Discriminator Loss: 0.1074, Generator Loss: 5.4657
2/2 [==============================] - 0s 66ms/step
Epoch: 7, Discriminator Loss: 1.0900, Generator Loss: 5.1275
2/2 [==============================] - 0s 72ms/step
Epoch: 8, Discriminator Loss: 0.8198, Generator Loss: 1.5542
2/2 [==============================] - 0s 70ms/step
Epoch: 9, Discriminator Loss: 0.3972, Ge

In [14]:
# Specify the path to save the models
generator.save("generator_model.h5")
discriminator.save("discriminator_model.h5")
gan.save("gan_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:

!cp generator_model.h5 /content/drive/MyDrive/


In [ ]:
import tensorflow as tf
import numpy as np

# Paths to the saved models
generator_path = 'generator_model.h5'
discriminator_path = 'discriminator_model.h5'

# Load the previously saved generator and discriminator models
generator = tf.keras.models.load_model(generator_path)
discriminator = tf.keras.models.load_model(discriminator_path)

# Ensure the discriminator is non-trainable while training the GAN
discriminator.trainable = False

# Re-create the GAN model
latent_dim = 100  # Same latent dimension used during training
gan_input = tf.keras.Input(shape=(latent_dim,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)

# Compile the GAN
gan = tf.keras.models.load_model("gan_model.h5")
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Function to continue training the GAN
def continue_training_gan(gan, generator, discriminator, dataset, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        # Train the Discriminator
        idx = np.random.randint(0, dataset.shape[0], batch_size)
        real_images = dataset[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)

        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print the progress and save the models periodically
        print("Epoch: {}, Discriminator Loss: {:.4f}, Generator Loss: {:.4f}".format(epoch, d_loss[0], g_loss))
        generate_and_save_images(generator, epoch)
        if epoch % 100 == 0:
            # Save the models
            generator.save('generator_model.h5')
            discriminator.save('discriminator_model.h5')
            gan.save('gan_model.h5')

# Parameters for training
EPOCHS = 1000
BATCH_SIZE = 64
continue_training_gan(gan, generator, discriminator, cat_images, latent_dim, EPOCHS, BATCH_SIZE)
